In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://labor.ny.gov/app/warn/default.asp?warnYr=2015"
url

'https://labor.ny.gov/app/warn/default.asp?warnYr=2015'

In [5]:
headers = {'accept-encoding': 'deflate'}
headers

{'accept-encoding': 'deflate'}

In [6]:
response = requests.get(url, headers=headers)
response

<Response [200]>

In [7]:
response.status_code

200

In [8]:
type(response.text)

str

In [9]:
soup = BeautifulSoup (response.text, 'html.parser')

In [19]:
table = soup.find("table")

In [20]:
rows =soup.find_all("tr")

In [21]:
len(rows)

434

In [31]:
# indicate the url start of the individual case URLs as they aren't the same as the launching page
newurl = "https://labor.ny.gov/app/warn/"

In [47]:
results = []
for row in rows:
    #print(row)
    a = row.find("a")['href']
    #print(a)
    company_url = f'{newurl}{a}'
    print(company_url)
    company_response = requests.get(company_url, headers=headers)
   # print (company_response.status_code)
    company_soup = BeautifulSoup(company_response.text, 'html.parser') 
   # print(company_soup)
    company_table = company_soup.find("table")
    # print(company_table)
    paragraphs = company_table.find_all("p")
    for p in paragraphs: 
        text=p.get_text('\n')
        if ('Date of Notice:' in text):
            notice_date = text.split(":")[1].strip().split('\n')[0].split()[0].replace(',', '').replace(';', '')
           # print(notice_date)
        elif ('Reason Stated for Filing:' in text):
            reason = text.split(":")[1].strip()
           # print(reason)
        elif ('Company:' in text):
            #split_company = text.split('\n')
           # company = split_company[1].strip()
            address = " ".join(split_company[3:])
            #print(company)
            #print(address)
        elif ('Phone:' in text):
            phone = text.split(":")[1].strip()
            #print(phone)
        elif ('Business Type:' in text):
            business_type = text.split(":")[1].strip()
            #print(business_type)
        elif ('Number Affected:' in text):
            affected = text.split(":")[1].strip().split()[0].replace(',', '')
            #print(affected)
        elif ('Total Employees:' in text):
            total_employees = text.split(":")[1].strip()
            #print(total_employees)
        elif ('Layoff Date:' in text):
            layoff_date = text.split(":")[1].strip()
            #print(layoff_date)
        elif ('Reason for Dislocation:' in text):
            dislocation = text.split(":")[1].strip()
            #print(dislocation)
        elif ('Union:' in text):
            union = text.split(":")[1].strip()
            #print(union)
        elif ('Classification:' in text):
            classification = text.split(":")[1].strip()
            #print(classification)
    result = {
        'notice_date': notice_date,
        'reason': reason,
        'company': company,
        'address': address,
        'phone': phone,
        'business_type': business_type,
        'affected': affected,
        'total_employees': total_employees,
        'layoff_date': layoff_date,
        'dislocation': dislocation,
        'union': union,
        'classification': classification
     }       
    results.append(result) 
    #break

https://labor.ny.gov/app/warn/details.asp?id=5398
https://labor.ny.gov/app/warn/details.asp?id=5393
https://labor.ny.gov/app/warn/details.asp?id=5394
https://labor.ny.gov/app/warn/details.asp?id=5396
https://labor.ny.gov/app/warn/details.asp?id=5397
https://labor.ny.gov/app/warn/details.asp?id=5391
https://labor.ny.gov/app/warn/details.asp?id=5390
https://labor.ny.gov/app/warn/details.asp?id=5389
https://labor.ny.gov/app/warn/details.asp?id=5385
https://labor.ny.gov/app/warn/details.asp?id=5387
https://labor.ny.gov/app/warn/details.asp?id=5382
https://labor.ny.gov/app/warn/details.asp?id=5383
https://labor.ny.gov/app/warn/details.asp?id=5384
https://labor.ny.gov/app/warn/details.asp?id=5381
https://labor.ny.gov/app/warn/details.asp?id=5378
https://labor.ny.gov/app/warn/details.asp?id=5379
https://labor.ny.gov/app/warn/details.asp?id=5380
https://labor.ny.gov/app/warn/details.asp?id=5375
https://labor.ny.gov/app/warn/details.asp?id=5376
https://labor.ny.gov/app/warn/details.asp?id=5372


In [48]:
df= pd.DataFrame(results)
df.head()

,notice_date,reason,company,address,phone,business_type,affected,total_employees,layoff_date,dislocation,union,classification
0,12/11/2015,Plant Layoff,Eastman Kodak Company (Kodak Office),"Rochester, NY 14650",(585) 781-1758,Photographic equipment and Supplies,1,-----,3/9/2016,Economic,The affected worker is not represented by a u...,Plant Layoff
1,1/28/2015,Plant Layoff,Eastman Kodak Company (Kodak Office),"Rochester, NY 14650",(803) 578-8188,Publishing,46,-----,to occur between 5/1/2015 and 7/31/2015,Economic,There are no bumping rights at this facility a...,Plant Layoff
2,12/1/2015,Plant Layoff,Eastman Kodak Company (Kodak Office),"Rochester, NY 14650",(212) 538-7303,Financial Institution,307,-----,to occur between 12/1/2015 through 3/31/2016,Economic,"No bumping rights exist, and employees are not...",Plant Layoff
3,12/18/2015,Plant Closing,Eastman Kodak Company (Kodak Office),"Rochester, NY 14650",(315) 349-2369,Power Plant,72,72,to occur between 3/31/2016 and 6/30/2016,Economic,International Brotherhood of Electrical Worker...,Plant Closing
4,12/18/2015,Plant Layoff,Eastman Kodak Company (Kodak Office),"Rochester, NY 14650",(585) 781-1758,Photographic equipment and supplies,1,-----,3/15/2016,Economic,Bumping rights do not exist.,Plant Layoff


In [50]:
df.to_csv('../data/2015.csv', index=False)